In [15]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.nn.modules import CrossEntropyLoss

from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score

from utils import ShopeeTrainDataset, ShopeeImageDataset, DistancePredict, get_metric, ShopeeScheduler

import timm

import os
from tqdm.notebook import tqdm
import math

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [2]:
class config:
    PATH = "./shopee-product-matching/"
    
    epoch = 15
    batch_size = 16
    num_workers=8
    prefetch_factor =8
    report_every_batch = 50
    
    scheduler_params = {
        "lr_start": 1e-5,
        "lr_max": 1e-5 * batch_size,     # 1e-5 * 32 (if batch_size(=32) is different then)
        "lr_min": 1e-6,
        "lr_ramp_ep": 5,
        "lr_sus_ep": 2,
        "lr_decay": 0.8,
    }
    arcface_scheduler_params = {
        "lr_start": 1e-4,
        "lr_max": 1e-4 * batch_size,     # 1e-5 * 32 (if batch_size(=32) is different then)
        "lr_min": 1e-5,
        "lr_ramp_ep": 5,
        "lr_sus_ep": 2,
        "lr_decay": 0.8,
    }

In [3]:
class ArcFace(nn.Module):
    """ NN module for projecting extracted embeddings onto the sphere surface """
    
    def __init__(self, in_features, out_features, s=30, m=0.5):
        super(ArcFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.arc_min = math.cos(math.pi - self.m)
        self.margin_min = math.sin(math.pi - self.m) * self.m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
    
    def _update_margin(self, new_margin):
        self.m = new_margin
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.arc_min = math.cos(math.pi - self.m)
        self.margin_min = math.sin(math.pi - self.m) * self.m

    def forward(self, embedding, label):
        cos = F.linear(F.normalize(embedding), F.normalize(self.weight))
        sin = torch.sqrt(1.0 - torch.pow(cos, 2)).clamp(0, 1)
        phi = cos * self.cos_m - sin * self.sin_m
        phi = torch.where(cos > self.arc_min, phi, cos - self.margin_min)

        one_hot = torch.zeros(cos.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        logits = one_hot * phi + (1.0 - one_hot) * cos
        logits *= self.s
        return logits

In [ ]:
class Model(nn.Module):
    def __init__(self, model_name, n_classes, margin=0.5, fc_dim=1024):
        super(Model, self).__init__()
        print("Building Model Backbone for {} model".format(model_name))
        self.model_name = model_name
        self.backbone = timm.create_model(model_name, pretrained=True)
        
        if "eca_nfnet" in model_name:
            feat_size = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
                
        elif "efficientnet" in model_name:
            feat_size = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        self.pooling =  nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.1)
        self.fc = nn.Linear(feat_size, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self.margin = ArcFace(fc_dim, n_classes, m=margin)
        self._init_params()

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x, labels=None):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)
        
        x = self.dropout(x)
        x = self.fc(x)
        x = self.bn(x)
        x = F.normalize(x,dim=1)
        if labels is not None:
            return self.margin(x,labels)
        else:
            return x

In [5]:
def read_dataset(name="train"):
    assert name in {"train", "test"}
    df = pd.read_csv(config.PATH + '{}.csv'.format(name))
    df["image_path"] = config.PATH + '{}_images/'.format(name) + df['image']

    return df

In [6]:
train = read_dataset("train")
label_group_dict = train.groupby("label_group").posting_id.agg("unique").to_dict()
train['target'] = train.label_group.map(label_group_dict)
train.head()


,posting_id,image,image_phash,title,label_group,image_path,target
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,./shopee-product-matching/train_images/0000a68...,"[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,./shopee-product-matching/train_images/0003978...,"[train_3386243561, train_3423213080]"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,./shopee-product-matching/train_images/000a190...,"[train_2288590299, train_3803689425]"
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,./shopee-product-matching/train_images/00117e4...,"[train_2406599165, train_3342059966]"
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,./shopee-product-matching/train_images/00136d1...,"[train_3369186413, train_921438619]"


In [7]:
n_classes = len(train["label_group"].unique())
num = int(0.2 * n_classes)
np.random.seed(1)
test_group = np.random.choice(train["label_group"].unique(), num)
#test_group
df_test = train[train["label_group"].isin(test_group)]
df_train = train[~train["label_group"].isin(test_group)]
print(len(df_train), len(df_test))

28194 6056


In [8]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = ShopeeTrainDataset(df_train, transform = transform)
test_dataset = ShopeeTrainDataset(df_test, transform = transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

d:\Repository\Kaggle\Shopee\utils.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['label_class'] = self.df['label_group'].map(class_mapping)


In [9]:
n_classes = len(train_dataset.df['label_group'].unique())
n_classes

9024

In [12]:
def get_image_feature(model, dataloader):
    image_features = []
    with torch.no_grad():
        for (images, labels) in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            features = model(images)
            image_features.append(features)
            del images, labels
    image_features = torch.cat(image_features, axis=0)

    torch.cuda.empty_cache()   
    return image_features

def validate(feature, threshold, df):
    pred = DistancePredict(df, feature, threshold= threshold)
    df["pred"] = pred
    f1, prec, rec = get_metric(df["target"], df["pred"])
    return f1, prec, rec

In [ ]:
# Training
def train(model_name, max_epochs, threshold, margin_min, margin_max):
    if not os.path.exists("model/{model_name}".format(model_name=model_name)):
        os.makedirs("model/{model_name}".format(model_name=model_name))
        
    best_f1 = -float("inf")
    
    model = Model(model_name, n_classes=n_classes, margin=margin_min).to(device)
    backbone_params = model.backbone.parameters()
    backbone_params_id = list(map(id, model.backbone.parameters()))
    other_params = filter(lambda p: id(p) not in backbone_params_id, model.parameters())

    optimizer = optim.AdamW(backbone_params, lr=config.scheduler_params['lr_start'])
    scheduler = scheduler = ShopeeScheduler(optimizer, **config.scheduler_params)
    optimizer2 = optim.AdamW(other_params, lr=config.arcface_scheduler_params['lr_start'])
    scheduler2 = scheduler = ShopeeScheduler(optimizer2, **config.arcface_scheduler_params)

    criterion = CrossEntropyLoss()

    for epoch in range(1,max_epochs+1):
        margin = (margin_max - margin_min) / max_epochs * (epoch - 1) + margin_min
        model.margin._update_margin(margin)
        print("Epoch: {:4f}  margin: {:2f}".format(epoch, margin))
        # Train
        model.train()
        for batch_id, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)
            output = model(images, labels)
            loss = criterion(output, labels)
            optimizer.zero_grad()
            optimizer2.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer2.step()

            if batch_id % config.report_every_batch == 0:
                # Calculate the accuracy on the batch
                output = output.data.cpu().numpy()
                output = np.argmax(output, axis=1)
                labels = labels.data.cpu().numpy()
                accuracy = accuracy_score(labels, output)
                F1 = f1_score(labels, output, labels=list(set(labels).union(set(output))), average='macro')
                # Print info
                print('Epoch: {:4}   [ {:5}/{}  ({:3.0f}%) ] \t\t Train Loss: {:.6f}     Accuracy: {:.3f}     F1 Score: {:.3f}'.format(
                    epoch,
                    batch_id * len(images), 
                    len(train_dataset), 
                    100. * batch_id / len(train_dataloader), 
                    loss.data,
                    accuracy,
                    F1,)
                )
            del images, labels, output, loss
            torch.cuda.empty_cache()

        scheduler.step()
        scheduler2.step()

        # Test and Save the model
        print("-----Epoch: {} Validation-----".format(epoch))
        # Test
        model.eval()
        image_features = get_image_feature(model, test_dataloader)
        f1, prec, rec = validate(image_features, threshold, df_test)
        if f1 > best_f1:
            torch.save(model.state_dict(), "model/{model_name}/{model_name}_margin_{margin}.pt".format(model_name = model_name, margin=margin))

        del image_features
        torch.cuda.empty_cache()

In [ ]:
margin_min = 0.2
margin_max = 0.95
train(model_name = "efficientnet_b4", max_epochs = config.epoch, threshold = 0.7, margin_min = margin_min, margin_max = margin_max)